# Load data via the API


Before running this notebook, initialize the database:

```
docker-compose exec web flask db init
```

In [4]:
import requests
import json
import os

In [5]:
BASE_URL = "http://web:8000/api/v0.0"
data_dir = os.getenv("DATA_DIR")

## Query for constants

In [6]:
response = requests.get(f'{BASE_URL}/entity_types')
response.raise_for_status()
response.json()

[['Radar', 'MeteoRadarMosaic', None],
 ['Station', 'PointWeatherObserver', None],
 ['Station', 'TemperatureMosaic', None],
 ['Station', 'EnergyConsumptionMonitor', None]]

## Load sources

In [7]:
with open(os.path.join(data_dir, 'sources.json')) as f:
    data = json.load(f)['sources']

In [11]:
response = requests.post(f'{BASE_URL}/sources', json=data)
response.raise_for_status()
tdmq_ids = response.json()

HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://web:8000/api/v0.0/sources

In [12]:
tdmq_ids

NameError: name 'tdmq_ids' is not defined

In [31]:
response = requests.get(f'{BASE_URL}/sources')
response.raise_for_status()
sources = response.json()

In [32]:
sources

[{'default_footprint': '{"type":"Point","coordinates":[9.22100000642642,30.0000000019687]}',
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_0',
  'tdmq_id': 'e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6'},
 {'default_footprint': '{"type":"Point","coordinates":[9.22200000642623,30.0030000019686]}',
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_1',
  'tdmq_id': '2bb83248-27f4-53c9-b149-ca74c8c283ee'},
 {'default_footprint': '{"type":"Point","coordinates":[9.55100000643269,32.0000000018894]}',
  'entity_category': 'Station',
  'entity_type': 'EnergyConsumptionMonitor',
  'external_id': 'tdm/sensor_3',
  'tdmq_id': '42608b02-a82a-58ae-84fb-f358bf9b7494'},
 {'default_footprint': '{"type":"Point","coordinates":[9.34300000645912,31.2000000019276]}',
  'entity_category': 'Station',
  'entity_type': 'PointWeatherObserver',
  'external_id': 'tdm/sensor_4',
  'tdmq_id': '8b53505d-79b8-548d-b160-

In [33]:
for s in sources:
    response = requests.get(f'{BASE_URL}/sources/{s["tdmq_id"]}')
    response.raise_for_status()
    print('*'*20)
    print(response.json())                       

********************
{'default_footprint': '{"type":"Point","coordinates":[9.22100000642642,30.0000000019687]}', 'description': {'alias': 'my desk', 'category': 'Station', 'controlledProperties': ['temperature', 'humidity'], 'default_footprint': {'coordinates': [9.221, 30.0], 'type': 'Point'}, 'description': {'brandName': 'ProSensor', 'manufacturerName': 'CRS4', 'modelName': 'R2D2', 'type': 'multisensor'}, 'id': 'tdm/sensor_0', 'stationary': True, 'type': 'PointWeatherObserver'}, 'entity_category': 'Station', 'entity_type': 'PointWeatherObserver', 'external_id': 'tdm/sensor_0', 'stationary': True, 'tdmq_id': 'e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6'}
********************
{'default_footprint': '{"type":"Point","coordinates":[9.22200000642623,30.0030000019686]}', 'description': {'alias': 'B221', 'category': 'Station', 'controlledProperties': ['humidity', 'temperature'], 'default_footprint': {'coordinates': [9.222, 30.003], 'type': 'Point'}, 'description': {'brandName': 'ProSensor', 'manufac

## Load records

In [16]:
with open(os.path.join(data_dir, 'records.json')) as f:
    data = json.load(f)['records']

In [17]:
response = requests.post(f'{BASE_URL}/records', json=data)
response.raise_for_status()
tdmq_ids = response.json()

## Run some queries

In [18]:
def query(url):
    response = requests.get(f'{BASE_URL}{url}')
    response.raise_for_status()
    print(json.dumps(response.json(), indent=4))

In [19]:
query('/sources?id=tdm/sensor_3')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.55100000643269,32.0000000018894]}",
        "entity_category": "Station",
        "entity_type": "EnergyConsumptionMonitor",
        "external_id": "tdm/sensor_3",
        "tdmq_id": "42608b02-a82a-58ae-84fb-f358bf9b7494"
    }
]


In [20]:
query('/sources?category=Station')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22100000642642,30.0000000019687]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22200000642623,30.0030000019686]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.55100000643269,32.0000000018894]}",
        "entity_category": "Station",
        "entity_type": "EnergyConsumptionMonitor",
        "external_id": "tdm/sensor_3",
        "tdmq_id": "42608b02-a82a-58ae-84fb-f358bf9b7494"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.34300000645912,31.2

In [21]:
query('/sources?type=EnergyConsumptionMonitor')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.55100000643269,32.0000000018894]}",
        "entity_category": "Station",
        "entity_type": "EnergyConsumptionMonitor",
        "external_id": "tdm/sensor_3",
        "tdmq_id": "42608b02-a82a-58ae-84fb-f358bf9b7494"
    }
]


In [22]:
query('/sources?controlledProperties=temperature')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22100000642642,30.0000000019687]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22200000642623,30.0030000019686]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.34300000645912,31.2000000019276]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_4",
        "tdmq_id": "8b53505d-79b8-548d-b160-777d116cb6b2"
    }
]


In [23]:
query('/sources?controlledProperties=temperature,humidity')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22100000642642,30.0000000019687]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22200000642623,30.0030000019686]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    }
]


In [24]:
query('/sources?after=2019-05-02T11:00:20Z')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22100000642642,30.0000000019687]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    }
]


In [25]:
query('/sources?stationary=false')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.11900000640193,29.0000000019969]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_5",
        "tdmq_id": "5df07546-b1d6-517f-8798-5abd79e7ebb1"
    }
]


In [26]:
query('/sources?stationary=true')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22100000642642,30.0000000019687]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_0",
        "tdmq_id": "e5a2f652-ea6f-5139-8bb3-b7a69f1b94c6"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.22200000642623,30.0030000019686]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_1",
        "tdmq_id": "2bb83248-27f4-53c9-b149-ca74c8c283ee"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.55100000643269,32.0000000018894]}",
        "entity_category": "Station",
        "entity_type": "EnergyConsumptionMonitor",
        "external_id": "tdm/sensor_3",
        "tdmq_id": "42608b02-a82a-58ae-84fb-f358bf9b7494"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.34300000645912,31.2

In [27]:
query('/sources?brandName=Acme')

[
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.34300000645912,31.2000000019276]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_4",
        "tdmq_id": "8b53505d-79b8-548d-b160-777d116cb6b2"
    },
    {
        "default_footprint": "{\"type\":\"Point\",\"coordinates\":[9.11900000640193,29.0000000019969]}",
        "entity_category": "Station",
        "entity_type": "PointWeatherObserver",
        "external_id": "tdm/sensor_5",
        "tdmq_id": "5df07546-b1d6-517f-8798-5abd79e7ebb1"
    }
]


In [35]:
for s in sources:
    response = requests.delete(f'{BASE_URL}/sources/{s["tdmq_id"]}')
    response.raise_for_status()

In [36]:
response = requests.get(f'{BASE_URL}/sources')
response.raise_for_status()
response.json()

[]